In [ ]:
!pip install -r ../requirements.txt

In [ ]:
# inference_demo.ipynb

# ## FarExStance Demo Inference Notebook
# 
# This notebook demonstrates running the XLM-R model for predicting stance and generating explanations
# on a small 5-sample dataset. All analyses and plots are included in the notebook.

# ### 1. Import Libraries
import torch
import pandas as pd
from torch.utils.data import DataLoader
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import sys

root_path = Path("..").resolve()
sys.path.append(str(root_path))

# Import repo modules
from common.dataset import DatasetInitModel, StanceDataset
from common.preprocessor import Preprocessor
from common.torch_trainer import TorchTrainer
from common.load_transformers import LoadTransformers
from models_architecture.TransformerWithExtraFeatures import TransformerWithExtraFeatures

# Logging function
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
def print_log(*args):
    for arg in args:
        print(arg)
        logger.info(arg)

# ### 2. Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_log(f"Device: {device}")

# ### 3. Model and Dataset Parameters
ner_model_id = 'HooshvareLab/bert-base-parsbert-ner-uncased'
dataset_labels = {'claim_name': "claim", 'text_name': "content",'label_name': "stance"}

dataset_params = {
    'padding' : True, 'truncation': True, 'max_length': 512,
    'claim_max_length': 38,
    'device': device,
    'similarity_features_no_for_start': 5,
    'similarity_features_no_for_end': 2,
    'ner_features_no': 8,
    'remove_dissimilar_sentences': True,
    'similar_sentence_no': 8,
    'ner_model_path': ner_model_id,
    'ner_tokenizer_path': ner_model_id,
    'ner_none_token_id': 0,
    'sentence_similarity_model_path': "sentence-transformers/all-MiniLM-L12-v2",
    'log_function': print_log,
    'save_load_features_path': 'demo/data/features',
    'remove_news_agency_name': False
}

transformer_params = {
    'model_loader': LoadTransformers.xlm_roberta,
    'transformer_model_path': "FacebookAI/xlm-roberta-base"
}

from transformers import AutoTokenizer
vectorizer = AutoTokenizer.from_pretrained(transformer_params["transformer_model_path"])
transformer_params["vectorizer"] = vectorizer

batch_size = 2
accumulation_steps = 1

# ### 4. Load Small 5-Sample Dataset
demo_dataset_path = "/home/kourosh/FarExStance/demo/data/new_test.xlsx"
df_demo = pd.read_excel(demo_dataset_path)
df_demo.head()

# ### 5. Create Dataset and DataLoader
test_dataset_input = {'data_set_path': demo_dataset_path, 'preprocessor': Preprocessor()}

init_params = DatasetInitModel(
    **test_dataset_input,
    **dataset_labels,
    **dataset_params,
    **transformer_params
)

test_dataset = StanceDataset(init_params)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print_log(f"Test dataset length: {len(test_dataloader.dataset)}")

# ### 6. Load Model
loss_fn = torch.nn.CrossEntropyLoss()
trainer = TorchTrainer(None, None, None, None, loss_fn, device, accumulation_steps, print_log)

model_save_directory = "models/"
epoch_model_name = "/epoch5_loss1.3054144382476807.pt"
trainer.test_dataloader = test_dataloader

loss, result_metrics, pred_labels = trainer.test_model(model_save_directory + epoch_model_name)

print_log("Loss:", loss)
print_log("Result metrics:", result_metrics)

# ### 7. Save Outputs
df_demo["pred_stance"] = pred_labels
df_demo["gen_explanation"] = test_dataset.explanation

result_save_path = "demo/results/"
Path(result_save_path).mkdir(parents=True, exist_ok=True)
df_demo.to_excel(f"{result_save_path}demo_results.xlsx", index=False)
df_demo.head()

# ### 8. Analysis and Visualization
# Plot the distribution of original stance labels
plt.figure(figsize=(6,4))
sns.countplot(data=df_demo, x="stance")
plt.title("Distribution of Original Labels (stance)")
plt.show()

# Plot the distribution of predicted stance labels
plt.figure(figsize=(6,4))
sns.countplot(data=df_demo, x="pred_stance")
plt.title("Distribution of Model Predictions")
plt.show()

# Confusion matrix comparing true vs predicted labels
confusion_matrix = pd.crosstab(df_demo['stance'], df_demo['pred_stance'], rownames=['Actual'], colnames=['Predicted'])
plt.figure(figsize=(6,5))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# Plot histogram of explanation lengths
df_demo['explanation_len'] = df_demo['gen_explanation'].apply(lambda x: len(str(x).split()))
plt.figure(figsize=(6,4))
sns.histplot(df_demo['explanation_len'], bins=5)
plt.title("Distribution of Generated Explanation Length (words)")
plt.show()


Device: cuda


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
IOStream.flush timed out


* if you have a good connection it will run